In [ ]:
# import pandas as pd
# import numpy as np
# import time
# import datetime
# from gnews import GNews
# import nltk
# from newspaper import Article
# from procedures import connect_to_db, insert_dataframe

# def get_dailynews_df(keyword):
    
#     # Set the parameters for the Google News API
#     google_news = GNews(language='id', country="Indonesia", max_results=100)
#     today = datetime.datetime.now().date()
#     google_news.start_date = (today - datetime.timedelta(days=2)).strftime("%Y-%m-%d")
#     google_news.end_date = today.strftime("%Y-%m-%d")
    
#     get_news = google_news.get_news(keyword)
    
#     news_df = pd.DataFrame(get_news)

#     # Check if the necessary columns are present in the DataFrame
#     column_names = ["publisher", "title", "description", "published date", "url"]
#     if all(col in news_df.columns for col in column_names):
#         news_df = news_df[column_names]
#     else:
#         return None

#     articles = []
#     images = []
    
#     for column, url in news_df["url"].items():
#         try:
#             # Download the article content and extract the text
#             # get_article = google_news.get_full_article(url)
#             # full_text = get_article.text
#             get_article = Article(url)
#             get_article.download()
#             get_article.parse()
            
#             full_text = get_article.text
            
#             # Try to get the image URL and append it to the list
#             if len(get_article.images) > 0:
#                 image = list(get_article.images)[0] if get_article.images else None
#                 images.append(image)
#             else:
#                 images.append(None)
            
#             articles.append(full_text)

#         except Exception as e:
#             # If there is an error, append an error message to the summary list and set the image URL to None
#             print(f"Error downloading article from {url}: {e}")
#             articles.append("Error: article download failed")
#             images.append(None)

#         # Sleep for a short time to avoid being blocked by the website
#         time.sleep(1)

#     # Add the full text and image URL columns to the DataFrame
#     news_df["isiartikel"] = articles
#     news_df['gambar'] = images

#     # Clean the full text column
#     news_df["isiartikel"] = news_df["isiartikel"].replace("\n", "", regex=True)
#     news_df["isiartikel"] = news_df["isiartikel"].replace("\r", "", regex=True)
#     news_df["isiartikel"] = news_df["isiartikel"].replace(r"\\", "", regex=True)

#     # Rename the columns and select the desired columns for the final DataFrame
#     news_df = news_df.rename(columns={"published date":"waktuterbit", "title":"judulartikel", "publisher":"sumberartikel"})
#     news_df = news_df[["sumberartikel", "judulartikel", "isiartikel", "url", "waktuterbit", "gambar"]]
#     news_df["sumberartikel"] = news_df["sumberartikel"].apply(lambda x: x["title"])
    
#     # Convert the "WaktuTerbit" column to datetime and extract date, month, and year
#     news_df["waktuterbit"] = pd.to_datetime(news_df["waktuterbit"])
#     news_df["tanggalterbit"] = news_df["waktuterbit"].dt.date
#     news_df["bulanterbit"] = news_df["waktuterbit"].dt.month_name()
#     news_df["tahunterbit"] = news_df["waktuterbit"].dt.year

#     return news_df


# pemilu_df = get_dailynews_df("pemilu")

# connection = connect_to_db()
# if connection:
#     insert_dataframe(connection, pemilu_df)
#     connection.close()


In [3]:
import pandas as pd
import numpy as np
import time
import datetime
from gnews import GNews
import nltk
from newspaper import Article

In [5]:
def get_articles(keyword: str, max_results: int, day_range: int) -> pd.DataFrame:

    # Set the parameters for the Google News API
    google_news = GNews(language='id', country="Indonesia", max_results=max_results)
    today = datetime.datetime.now().date()
    
    start_date = (today - datetime.timedelta(days=day_range)).strftime("%Y-%m-%d")
    end_date = today.strftime("%Y-%m-%d")


    google_news.start_date = start_date
    google_news.end_date = end_date

    get_news = google_news.get_news(keyword)

    articles_df = pd.DataFrame(get_news)

    # Check if the necessary columns are present in the DataFrame
    column_names = ["publisher", "title", "description", "published date", "url"]
    if all(col in articles_df.columns for col in column_names):
        articles_df = articles_df[column_names]
    else:
        return None

    articles = []
    images = []

    for column, url in articles_df["url"].items():
        try:
            # Download the article content and extract the text
            # get_article = google_news.get_full_article(url)
            # full_text = get_article.text
            get_article = Article(url)
            get_article.download()
            get_article.parse()
    
            full_text = get_article.text
    
            # Try to get the image URL and append it to the list
            if len(get_article.images) > 0:
                image = list(get_article.images)[0] if get_article.images else None
                images.append(image)
            else:
                images.append(None)
    
            articles.append(full_text)

        except Exception as e:
            # If there is an error, append an error message to the summary list and set the image URL to None
            print(f"Error downloading article from {url}: {e}")
            articles.append("Error: article download failed")
            images.append(None)

        # Sleep for a short time to avoid being blocked by the website
        time.sleep(1)

    # Add the full text and image URL columns to the DataFrame
    articles_df["article_text"] = articles
    articles_df['image'] = images

    # Clean the full text column
    articles_df["article_text"] = articles_df["article_text"].replace("\n", "", regex=True)
    articles_df["article_text"] = articles_df["article_text"].replace("\r", "", regex=True)
    articles_df["article_text"] = articles_df["article_text"].replace(r"\\", "", regex=True)

    # # Rename the columns and select the desired columns for the final DataFrame
    # articles_df = articles_df.rename(columns={"published date":"published_date", "title":"judulartikel", "publisher":"sumberartikel"})
    # articles_df = articles_df[["sumberartikel", "judulartikel", "isiartikel", "url", "waktuterbit", "gambar"]]
    # articles_df["sumberartikel"] = articles_df["sumberartikel"].apply(lambda x: x["title"])

    # # Convert the "WaktuTerbit" column to datetime and extract date, month, and year
    # articles_df["waktuterbit"] = pd.to_datetime(articles_df["waktuterbit"])
    # articles_df["tanggalterbit"] = articles_df["waktuterbit"].dt.date
    # articles_df["bulanterbit"] = articles_df["waktuterbit"].dt.month_name()
    # articles_df["tahunterbit"] = articles_df["waktuterbit"].dt.year

    return articles_df

In [6]:
my_df = get_articles("kebakaran hutan", 10, 2)

my_df

TypeError: unsupported operand type(s) for -: 'str' and 'str'